In [1]:
# Import libraries needed for webscraping
import requests
from datetime import datetime
import time
import random
import pandas as pd
from tqdm import tqdm

import praw
from psaw import PushshiftAPI

pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 500)

In [2]:
#url_1 = 'https://www.reddit.com/r/TravelHacks.json'
#url_2 = 'https://www.reddit.com/r/awardtravel.json'
#res = requests.get(url, headers={'User-agent': 'Helpful-Scientist-53'})

In [2]:
url = 'https://api.pushshift.io/reddit/search/submission'

In [ ]:
# url for subreddit submission 
# url = 'https://api.pushshift.io/reddit/search/submission?subreddit=travel'

In [3]:
# set up parameter
params = {
    'subreddit': 'travel',
    'size': 1000, # limit of 1000
    'before': 1680108395 # show the post before 1680108395  
}

In [4]:
# set up response
res = requests.get(url, params)

In [5]:
# check status code
res.status_code

200

In [6]:
# returned as json in dictionary list
data = res.json()

In [7]:
posts = data['data']

In [8]:
# size for the default value of 10; changed it to 1000 most recent posts
len(posts)

1000

In [9]:
# check the first post
posts[0]

# created_utc is an int, showing the time the first post weas posted

{'subreddit': 'travel',
 'selftext': "Never been before so dont know much  about the region. Where should I go that is affordable, interesting,  and has good food? I will have my GF with me so not interested in  cooming, alcohol, or nightlife. We're travelling in November because of  fewer crowds and cheaper prices. We know we want to visit Hungary for  sure for the thermal baths and the food but are unsure if we should go  north from there to Slovakia/Czechia/Poland, to Slovenia/Croatia/Serbia,  or east towards Romania. I was leaning towards Croatia and Serbia  because the weather may be warmer, the proximity to Hungary, I've heard  food there is decent, and I like Orthodox churches. But we are open to  anywhere accessible by train or bus from Budapest. Any good outdoor  things to do in any of these regions in November?",
 'author_fullname': 't2_3s3ea',
 'gilded': 0,
 'title': 'I will have two weeks of vacation in November and want to visit some ountries in eastern Europe.',
 'link_fl

In [10]:
# turned dictionary to dataframe
df = pd.DataFrame(posts)

In [11]:
df.head(1)

,subreddit,selftext,author_fullname,gilded,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,thumbnail_height,top_awarded_type,hide_score,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,total_awards_received,media_embed,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,score,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,link_flair_type,wls,removed_by_category,author_flair_type,domain,allow_live_comments,suggested_sort,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,link_flair_template_id,can_gild,spoiler,locked,author_flair_text,treatment_tags,removed_by,distinguished,subreddit_id,link_flair_background_color,id,is_robot_indexable,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,retrieved_utc,updated_utc,utc_datetime_str,post_hint,url_overridden_by_dest,preview,crosspost_parent_list,crosspost_parent,is_gallery,media_metadata,gallery_data,author_cakeday
0,travel,Never been before so dont know much about the...,t2_3s3ea,0,I will have two weeks of vacation in November ...,[],r/travel,False,6.0,question,NaN,None,True,False,light,1.0,None,public,0,{},NaN,None,False,None,False,False,None,{},Question,1,False,False,self,False,None,[],{},None,True,text,6.0,None,text,self.travel,False,None,None,False,True,True,False,False,[],[],False,63ab44d8-3640-11e3-8665-12313d21c4ee,True,False,False,None,[],None,None,t5_2qh41,#ff66ac,125tfj8,True,xss2,None,0,True,all_ads,False,False,None,/r/travel/comments/125tfj8/i_will_have_two_wee...,all_ads,False,https://www.reddit.com/r/travel/comments/125tf...,7443770,1680108205,0,None,False,1680108218,1680108218,2023-03-29 16:43:25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# showed the columns in interest
df[['subreddit', 'selftext', 'title']].head(1)

,subreddit,selftext,title
0,travel,Never been before so dont know much about the...,I will have two weeks of vacation in November ...


In [13]:
# create function for collecting data
# credit: https://www.youtube.com/watch?v=AcrjEWsMi_E and class note

# create instance for URL
url = 'https://api.pushshift.io/reddit/search/submission'

# loop n_requests iterations
def webScrape(subreddit, url, data_name, n_requests):  
    df_final =  pd.DataFrame(columns=['subreddit', 'selftext', 'title', 'distinguished', 'author',
                                      'removed_by_category', 'num_comments', 'utc_datetime_str'])
    for i in range(1,n_requests):                # set up request range based on 1000 post per run
        if i==1:
            params = {'subreddit': subreddit, 
                      'size': 1000}              # limit to 1000 posts per run
        else:
            params = {'subreddit': subreddit, 
                      'size': 1000,
                      'before': before}
        
        res = requests.get(url, params)          # set up response  
        if res.status_code != 200:               # check for errors and raise ConnectionError if there is one
            raise ConnectionError(f'Error code:{res.status_code} has occured.')
            
        data = res.json()
        posts = data['data']
        df = pd.DataFrame(posts)                 # turned dictionary to dataframe
        df = df[['subreddit', 'selftext', 'title', 'distinguished', 'author', 'removed_by_category', 'num_comments', 'utc_datetime_str']]  # showed the columns in interest
        df_final = pd.concat([df_final, df])         # combined the dataframe
        before = posts[-1]['created_utc']        # show the post before certain time
        time.sleep(3)                            # re-attempting collection after 3 sec if api is overloaded
    
    # drop duplicate rows that may have been pulled:
    df_final.drop_duplicates(subset='title', keep='first', inplace = True)
    
    #Save the DataFrame as a .csv file
    df_final.to_csv('../data/' + str(data_name) + '.csv', index = False, sep = ",")
    
    # print number of posts pulled
    print(f"Total of posts were downloaded: {df_final.shape[0]}")
    

### WebScrape for first subraddit: travel

In [104]:
#webScrape('travel', url, 'travel_post_2000', 3)

Total of posts were downloaded: 1930


In [105]:
#chcek_df = pd.read_csv('../data/travel_post_2000.csv')

In [97]:
#duplicated_title = chcek_df["title"].duplicated(keep=False)
#duplicated_selftext = chcek_df["selftext"].duplicated(keep=False)

In [98]:
#saving the index of identified duplicated rows
#title_index = chcek_df["title"][duplicated_title].sort_values().index
#selftext_index = chcek_df["selftext"][duplicated_selftext].sort_values().index

In [102]:
#print first 10 rows of duplicated rows
#chcek_df.loc[title_index].head(20)

,subreddit,selftext,title,link_flair_css_class,id,removed_by_category,domain,num_comments,created_utc,utc_datetime_str
1880,travel,NaN,10 Must-Visit Destinations for Adventure Trave...,NaN,121z9ku,reddit,thecomprehensiveblog.blogspot.com,0,1679778577,2023-03-25 21:09:37
1878,travel,[removed],10 Must-Visit Destinations for Adventure Trave...,NaN,121zc1j,automod_filtered,self.travel,0,1679778711,2023-03-25 21:11:51
1607,travel,[removed],4 Exhibitions To Warm Your Heart During Cold W...,advice,122u4di,automod_filtered,self.travel,0,1679853468,2023-03-26 17:57:48
1612,travel,NaN,4 Exhibitions To Warm Your Heart During Cold W...,article,122tspm,reddit,theistanbulchronicle.com,0,1679852785,2023-03-26 17:46:25
131,travel,NaN,A little Known Place in Japan-The Tottori Sand...,NaN,126tnvl,automod_filtered,self.travel,0,1680197425,2023-03-30 17:30:25
138,travel,NaN,A little Known Place in Japan-The Tottori Sand...,images,126sztv,moderator,i.redd.it,0,1680195923,2023-03-30 17:05:23
144,travel,NaN,ATA Safari Shop Plc – Arusha Travel Agency Ltd...,NaN,126s87n,moderator,i.redd.it,0,1680194239,2023-03-30 16:37:19
165,travel,[removed],ATA Safari Shop Plc – Arusha Travel Agency Ltd...,meta,126pein,moderator,self.travel,0,1680188023,2023-03-30 14:53:43
166,travel,NaN,ATA Safari Shop Plc – Arusha Travel Agency Ltd...,images,126p6r8,moderator,i.redd.it,0,1680187517,2023-03-30 14:45:17
167,travel,NaN,ATA Safari Shop Plc – Arusha Travel Agency Ltd...,images,126p60j,moderator,i.redd.it,0,1680187468,2023-03-30 14:44:28


In [101]:
#print first 10 rows of duplicated rows
#chcek_df.loc[selftext_index].tail(10)

,subreddit,selftext,title,link_flair_css_class,id,removed_by_category,domain,num_comments,created_utc,utc_datetime_str
1977,travel,NaN,Cultural experiences,article,121py3o,reddit,jtsofttech.com,0,1679759347,2023-03-25 15:49:07
1978,travel,NaN,The Connection Between Our Health and the Heal...,news,121pvft,reddit,jtsofttech.com,0,1679759182,2023-03-25 15:46:22
1981,travel,NaN,Virtual Walk on Dolphin Island Philippines,video,121plk5,moderator,youtu.be,0,1679758578,2023-03-25 15:36:18
1983,travel,NaN,Trip to Reykjavík | Iceland Travel Guide,NaN,121p8z7,reddit,bestplacevisit.com,0,1679757812,2023-03-25 15:23:32
1988,travel,NaN,Make Money Online Effortlessly with EarnEase,NaN,121or2y,reddit,share.earnease.net,0,1679756749,2023-03-25 15:05:49
1989,travel,NaN,I am interested in having a trip to Montana ma...,question,121onkb,automod_filtered,self.travel,0,1679756551,2023-03-25 15:02:31
1990,travel,NaN,You have one chance to make someone fall in lo...,advice,121ofz8,automod_filtered,self.travel,0,1679756209,2023-03-25 14:56:49
1993,travel,NaN,A neat pic of the moon from Bahamas,images,121o8zf,moderator,i.redd.it,0,1679755902,2023-03-25 14:51:42
1997,travel,NaN,I like to travel a lot. how do you like,NaN,121nlag,automod_filtered,self.travel,0,1679754822,2023-03-25 14:33:42
1999,travel,NaN,"Coliseo, Rome Italy",images,121nb94,automod_filtered,reddit.com,0,1679754213,2023-03-25 14:23:33


In [14]:
webScrape('travel', url, 'travel_post_4000', 5)

Total of posts were downloaded: 3871


In [15]:
webScrape('travel', url, 'travel_post_8000', 9)

Total of posts were downloaded: 7739


In [16]:
webScrape('travel', url, 'travel_post_20000', 21)

Total of posts were downloaded: 19354


In [17]:
travel_df1 = pd.read_csv('../data/travel_post_4000.csv')
travel_df2 = pd.read_csv('../data/travel_post_8000.csv')
travel_df = pd.read_csv('../data/travel_post_20000.csv')

In [18]:
travel_df1.shape, travel_df2.shape

((3871, 8), (7739, 8))

In [19]:
travel_df1.head()

,subreddit,selftext,title,distinguished,author,removed_by_category,num_comments,utc_datetime_str
0,travel,NaN,Long way back home!,NaN,Kaydno419,moderator,0,2023-04-02 07:02:34
1,travel,Planning a trip for 4 nights in Corfu and was ...,How expensive is Corfu?,NaN,Nikiad,NaN,0,2023-04-02 06:56:25
2,travel,I’ve heard that if you go to Costa Rica withou...,Winging it in Costa Rica,NaN,lucylovesugly1,NaN,1,2023-04-02 06:53:38
3,travel,My husband and I can’t decide where to go for ...,Anniversary trip ideas in the US,NaN,HelloKiks,NaN,0,2023-04-02 06:49:24
4,travel,[removed],Road Trip Ideas For The Pacific Coast Highway,NaN,Ocheson360,reddit,0,2023-04-02 06:44:36


In [20]:
travel_df2.tail()

,subreddit,selftext,title,distinguished,author,removed_by_category,num_comments,utc_datetime_str
7734,travel,[removed],Holidays to Turkey in 2023,NaN,maellecampion,reddit,1,2023-03-07 18:55:01
7735,travel,NaN,Possible Danger?,NaN,Reasonable_Dog1652,moderator,0,2023-03-07 18:50:53
7736,travel,NaN,Wanting to continue to feed my soul and spirit...,NaN,prestonsouthwind,reddit,0,2023-03-07 18:47:57
7737,travel,NaN,What do y’all recommend me to do or where to g...,NaN,shelltepin,automod_filtered,0,2023-03-07 18:34:36
7738,travel,NaN,"Fundraiser for Adventuring, Connecting with Na...",NaN,prestonsouthwind,reddit,0,2023-03-07 18:30:35


In [21]:
travel_df.tail()

,subreddit,selftext,title,distinguished,author,removed_by_category,num_comments,utc_datetime_str
19349,travel,"Ok, I have two passports. Here’s my question:\...",Can I get a visa on my second passport after m...,NaN,Suitable_Ad_7564,NaN,0,2023-01-23 23:31:12
19350,travel,[removed],Kagoshima taxis,NaN,Louisemariette,reddit,0,2023-01-23 23:27:38
19351,travel,"Hello, I will be travling to Lithuania and sta...",Travel Insurance for Lithuanian Visa,NaN,Sepabo,NaN,0,2023-01-23 23:24:16
19352,travel,NaN,We traveled across the country in 4 days!!!! …...,NaN,Flimsy-Abrocoma9682,moderator,0,2023-01-23 23:22:34
19353,travel,NaN,Schönbrunn Palace Park/ Schönbrunner Schlosspa...,NaN,Peripatetic_Virgo,automod_filtered,0,2023-01-23 23:21:59


In [22]:
travel_df.keys()

Index(['subreddit', 'selftext', 'title', 'distinguished', 'author',
       'removed_by_category', 'num_comments', 'utc_datetime_str'],
      dtype='object')

In [23]:
travel_df["selftext"].value_counts()

[removed]                                                                                                                                                                                                                                                                                                                                                                                                 5060
[deleted]                                                                                                                                                                                                                                                                                                                                                                                                   10
Any info or advice on how to better research this question is greatly appreciated!                                                                                                                        

### WebScrape for second subraddit: awardtravel

In [24]:
webScrape('awardtravel', url, 'awardtravel_4000', 5)

Total of posts were downloaded: 3951


In [25]:
webScrape('awardtravel', url, 'awardtravel_8000', 9)

Total of posts were downloaded: 7777


In [26]:
webScrape('awardtravel', url, 'awardtravel_12000', 13)

Total of posts were downloaded: 11540


In [27]:
awardtravel_df1 = pd.read_csv('../data/awardtravel_4000.csv')
awardtravel_df2 = pd.read_csv('../data/awardtravel_8000.csv')
awardtravel_df = pd.read_csv('../data/awardtravel_12000.csv')

In [28]:
awardtravel_df1.keys()

Index(['subreddit', 'selftext', 'title', 'distinguished', 'author',
       'removed_by_category', 'num_comments', 'utc_datetime_str'],
      dtype='object')

In [29]:
awardtravel_df.tail()

,subreddit,selftext,title,distinguished,author,removed_by_category,num_comments,utc_datetime_str
11535,awardtravel,Are there any self parking structures around t...,Parking around Waldorf Astoria Beverly Hills h...,NaN,acivicbboi,NaN,4,2020-10-02 00:11:41
11536,awardtravel,I booked the Thompson Chicago with points this...,Hyatt no parking available point rebate,NaN,xcoldricex,NaN,18,2020-10-01 22:09:09
11537,awardtravel,Any recommendations? I have Ziva booked for 6...,"Ziva Cabo vs The Cape, a Thompson Hotel",NaN,sosamo11,NaN,37,2020-10-01 16:52:05
11538,awardtravel,"I have 60K Lifemiles, and do not plan to trave...",Lifemiles,NaN,sosamo11,NaN,6,2020-10-01 16:00:08
11539,awardtravel,[deleted],What Award Program Should I get into,NaN,[deleted],deleted,0,2020-10-01 12:58:40


In [30]:
awardtravel_df1.shape, awardtravel_df2.shape

((3951, 8), (7777, 8))

### WebScrape for third subraddit: backpacking

In [31]:
webScrape('backpacking', url, 'backpacking_4000', 5)

Total of posts were downloaded: 3752


In [32]:
webScrape('backpacking', url, 'backpacking_8000', 9)

Total of posts were downloaded: 7455


In [33]:
webScrape('backpacking', url, 'backpacking_12000', 13)

Total of posts were downloaded: 11178


In [34]:
backpacking_df1 = pd.read_csv('../data/backpacking_4000.csv')
backpacking_df2 = pd.read_csv('../data/backpacking_8000.csv')
backpacking_df = pd.read_csv('../data/backpacking_12000.csv')

In [35]:
backpacking_df.tail()

,subreddit,selftext,title,distinguished,author,removed_by_category,num_comments,utc_datetime_str
11173,backpacking,NaN,The Annapurna Express अन्नपूर्ण एक्सप्रेस 🇳🇵,NaN,Globetrotter-Gary,moderator,1,2022-02-26 06:00:05
11174,backpacking,[removed],Anyone have any experience with Hanwag boots f...,NaN,BobPaulson99,moderator,1,2022-02-26 03:27:18
11175,backpacking,[removed],"Forcibly locked for three years, Guo Wengui to...",NaN,Competitive-Eye-5505,moderator,1,2022-02-26 02:34:25
11176,backpacking,NaN,Epic Underwater Trip - Must See,NaN,Professional-Sun132,NaN,5,2022-02-25 23:47:55
11177,backpacking,NaN,Working as a goatherd in the stunning and secl...,NaN,EssayOnAPostcard,moderator,14,2022-02-25 20:57:54


In [36]:
backpacking_df1.shape, backpacking_df2.shape

((3752, 8), (7455, 8))

### WebScrape for third subraddit: cruise

In [37]:
webScrape('cruise', url, 'cruise_4000', 5)

Total of posts were downloaded: 3946


In [38]:
webScrape('cruise', url, 'cruise_8000', 9)

Total of posts were downloaded: 7870


In [39]:
webScrape('cruise', url, 'cruise_12000', 13)

Total of posts were downloaded: 11414


In [40]:
cruise_df1 = pd.read_csv('../data/cruise_4000.csv')
cruise_df2 = pd.read_csv('../data/cruise_8000.csv')
cruise_df = pd.read_csv('../data/cruise_12000.csv')

In [41]:
cruise_df.tail()

,subreddit,selftext,title,distinguished,author,removed_by_category,num_comments,utc_datetime_str
11409,Cruise,[removed],How we are now witnessing the slippery slope t...,NaN,mrtrumbo,moderator,9,2021-08-22 21:08:56
11410,Cruise,NaN,"On the Carnival Vista, is there an electric so...",NaN,Khalnik,NaN,3,2021-08-22 19:36:35
11411,Cruise,[deleted],First Ever Cruise Felt Incredibly Disappointin...,NaN,[deleted],deleted,9,2021-08-22 18:32:23
11412,Cruise,NaN,Q: For large families (5 people) what have you...,NaN,gvlaks,NaN,10,2021-08-22 17:59:12
11413,Cruise,Anyone ever do the Atlantis day pass in Nassau...,Atlantis Day Pass?,NaN,wbuttersbutt,NaN,3,2021-08-22 17:49:47


### WebScrape for third subraddit: TravelHacks

In [42]:
webScrape('TravelHacks', url, 'TravelHacks_4000', 5)

Total of posts were downloaded: 3868


In [43]:
webScrape('TravelHacks', url, 'TravelHacks_8000', 9)

Total of posts were downloaded: 7744


In [44]:
webScrape('TravelHacks', url, 'TravelHacks_12000', 13)

Total of posts were downloaded: 11293
